# Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networks

## Abstract

图像到图像的转换是一类视觉和图形问题，其目标是使用对齐的图像对的训练集来学习输入图像和输出图像之间的映射。但是，对于许多任务，配对的训练数据将不可用。我们提出了一种学习方法，用于在没有配对实例的情况下将图像从源域X转换为目标域Y.我们的目标是学习映射G：X→Y，使得G（X）中的图像分布与使用对抗性损失的分布Y不可区分。因为这个映射是高度欠约束的，所以我们将它与一个逆映射F：Y→X耦合起来，引入一个循环一致性损失来强制F（G（X））≈X（反之亦然）。定性结果是在不存在配对训练数据（包括收集风格转换，对象变形，季节转移，照片增强等）的若干任务中呈现的。针对若干先前方法的定量比较证明了我们的方法的优越性。

## Introduction

1873年，一个可爱的春日，克劳德·莫奈在阿根廷附近塞纳河岸边放置画架时，看到了什么？（图1，左上角）？ 一张彩色照片，如果它被发明，可能已经记录了一片清脆的蓝天和反映它的玻璃状河流。 莫奈通过细小的笔触和明亮的调色板，传达了他对同一场景的印象。  

如果莫奈在一个凉爽的夏天夜晚在卡西斯的小港上发生了什么事（图1左下角）呢？ 在莫奈的一幅画廊中漫步，可以想象他将如何渲染这个场景：或许是以柔和的色调，突然的油漆和平滑的动态范围。  

我们可以想象，尽管从来没有在他所画的场景的照片旁边看到莫奈绘画的并排例子。 相反，我们知道莫奈绘画和一组风景照片。 我们可以推断这两组之间的风格差异，从而想象一下，如果我们将它从一套“翻译”成另一套，场景可能会是什么样子  

在本文中，我们提出了一种方法，可以学习做同样的事情：捕捉一个图像集合的特殊特征，并找出如何将这些特征转换为其他图像集合，所有在没有任何配对训练的例子。  

这个问题可以更广泛地描述为图像到图像的转换[21]，将图像从给定场景的一个表示x转换到另一个y，例如灰度转换为彩色，图像转换为语义标签，边缘图转换成照片。 在计算机视觉，图像处理，计算机摄影和图形方面的多年研究已经在监督设置中产生了强大的翻译系统，其中示例图像对{x，y}可用（例如，图2左），然而，获得配对训练 数据可能是困难的和昂贵的。 例如，对于诸如语义分割（例如[4]）的任务，仅存在几个数据集，并且它们相对较小。 由于期望的输出是非常复杂的，通常需要艺术创作，所以获得用于诸如艺术风格的图形任务的输入 - 输出对可能更加困难。 对于许多任务，如对象变形（例如斑马，图1中上），期望的输出甚至没有明确定义。  

因此，我们寻求一种算法，可以学习在没有配对输入输出示例的域之间进行转换（图2，右）。我们假设这些领域之间存在着某种潜在的关系 - 例如，它们是同一个基础场景的两种不同的效果图 - 并试图学习这种关系。虽然我们缺乏对偶实例的监督，但是我们可以在集合层面上利用监督：在域X中给予一组图像，在域Y给予不同的集合。我们可以训练一个映射G：X→Y，这样输出y = G（x），x∈X，y被y训练成对y进行分类的对手不可区分。从理论上讲，这个目标可以导致y的输出分布与经验分布pdata（y）相匹配（一般来说，这需要G是随机的）[15]。因此最佳的G将域X翻译成分布与Y相同的域Y.然而，这样的翻译并不能保证单个输入x和输出y是以有意义的方式配对的 - 有无穷多的映射G将导致y上的相同分布。而且，在实践中，我们发现很难孤立地优化敌对目标：标准程序通常会导致众所周知的模式崩溃问题，即所有输入图像都映射到相同的输出图像，并且优化未能取得进展[14] 。  

这些问题要求为我们的目标增加更多的结构。 因此，我们利用翻译应该是“循环一致”的性质，就是说，如果我们把一个从英语到法语的句子翻译过来，然后从法语翻译成英语，我们应该回到原来的句子[3]。 在数学上，如果我们有一个翻译器G：X→Y和另一个翻译器F：Y→X，那么G和F应该是彼此逆的，并且这两个映射应该是双射。 我们通过同时训练映射G和F来应用这个结构假设，并增加一个鼓励F（G（x））≈x和G（F（y））≈y的循环一致性损失[64]。 将这种损失与X领域和Y领域的对抗性损失相结合，就可以完全实现我们对不成对的图像翻译的目标。  

我们将我们的方法应用于广泛的应用，包括收藏风格转换，物体变形，季节转换和照片增强。 我们还将之前的方法与依靠手工定义的样式和内容因子或共享嵌入函数进行比较，并显示我们的方法优于这些基线。 我们的代码可以在https：// github上找到。COM/ junyanz/ CycleGAN。 请在= https：//junyanz.github.io/CycleGAN/查看更多结果。  

## Related work 

生成对抗网络（GANs）[15,62]在图像生成[5,37]，图像编辑[65]和表示学习[37,42,35]中取得了令人印象深刻的结果。 最近的方法对条件图像生成应用（如text2image [39]，图像修复[36]和未来预测[34]以及视频[53]和3D数据[56]等其他领域采用相同的思路。 GANs成功的关键在于对抗性损失的观念，这种损失迫使生成的图像原则上与真实图像无法区分。 这对于图像生成任务特别有用，因为这正是计算机图形学大部分旨在优化的目标。 我们采取对抗性的损失来学习映射，使得翻译后的图像不能与目标域中的图像区分开来。  

图像到图像的转换图像到图像转换的思想至少可以追溯到赫茨曼（Hertzmann）等人的图像类比（Image Analogies）[18]，他们在一个输入输出训练图像对上采用非参数化的纹理模型[9]。 更近期的方法使用输入 - 输出示例的数据集来学习使用CNN的参数翻译功能，例如，[31]。 我们的方法建立在Isola等人的“pix2pix”框架上。 [21]，它使用条件生成敌对网络[15]学习输入映射到输出图像。 类似的想法已经应用于各种任务，如从草图生成照片[43]或从属性和语义布局[23]。 然而，不像这些先前的作品，我们学习了没有配对训练实例的映射。  

不成对的图像到图像转换其他几种方法还处理不成对的设置，其目标是将两个数据域X和Y相关联。 Rosales等人[40]提出了一个贝叶斯框架，包括基于从源图像计算的基于斑块的马尔可夫随机场的先验和从多个样式图像获得的似然项。最近，CoGAN [30]和跨模态场景网络[1]使用权重分享策略来学习跨领域的共同表示。与我们的方法同时，刘等人。 [29]扩展了这个框架，结合了变分自动编码器[25]和生成对抗网络。另一行同时工作[45,48,2]鼓励输入和输出共享某些“内容”功能，即使它们可能在“风格”上有所不同。他们还使用对抗性网络，附加条款来强制输出接近预定义度量空间的输入，如类标签空间[2]，图像像素空间[45]和图像特征空间[48]。  

与上述方法不同，我们的公式并不依赖于输入和输出之间的任何特定的，预定义的相似性函数，也不假定输入和输出必须位于相同的低维嵌入空间中。 这使得我们的方法成为许多视觉和图形任务的通用解决方案。 我们直接比较了5.1节中的一些先前和当前的方法。  

循环一致性使用传递性作为调整结构化数据的方法的想法有着悠久的历史。在视觉追踪中，强制执行简单的前向一致性一直是几十年来的标准技巧[47]。在语言领域，通过“回译和重新翻译”来验证和改进翻译是人类翻译家[3]（包括幽默地由马克·吐温[50]）以及机器[16]所使用的技术。最近，运动[60]，三维形状匹配[20]，cosegmentation [54]，密集语义对齐[63,64]和深度估计[13]的结构中使用了高阶循环一致性。其中，周等人。 [64]和戈达尔等人。 [13]与我们的工作最相似，因为他们使用循环一致性损失作为使用传递性来监督CNN培训的一种方式。在这项工作中，我们引入了类似的损失，推动G和F相互一致。与我们的工作同时，在这些相同的程序中，Yi et al。 [58]独立地使用类似的目标进行不成对的图像到图像的翻译，受机器翻译双重学习的启发[16]。  

神经风格转移[12,22,51,11]是执行图像转换的另一种方式，它通过将一幅图像的内容与另一幅图像（通常是一幅绘画）的风格相结合来合成新颖的图像 预先训练的深度特征的克矩阵统计。 另一方面，我们的主要焦点是通过试图捕获更高级别的外观结构之间的对应关系来学习两个图像集合之间的映射，而不是两个特定图像之间的映射。 因此，我们的方法可以应用于其他任务，如绘画→照片，对象变形等单个样本传输方法不能很好地执行。 我们在5.2节比较这两种方法。  

## Formulation 

我们的目标是学习给定训练样本{xi} N i = 1的两个域X和Y之间的映射函数，其中xi∈X和{yj} M j = 1，其中yj∈Y 1。 我们将数据分布表示为x〜pdata（x）和y〜pdata（y）。 如图3（a）所示，我们的模型包括两个映射G：X→Y和F：Y→X。此外，我们引入两个敌对鉴别器DX和DY，其中DX旨在区分图像{x} 图像{F（y）}; 同样，DY旨在区分{y}和{G（x）}。 我们的目标包含两种类型的术语：对抗性损失[15]，用于匹配生成图像的分布与目标域中的数据分布; 和周期一致性损失，以防止学习到的映射G和F相互矛盾。  

对抗性损失  

我们将对抗损失[15]应用于两个映射函数。 对于映射函数G：X→Y及其鉴别器DY，我们将目标表示为：

公式

其中G试图产生看起来类似于来自域Y的图像的图像G（x），而DY旨在区分经翻译的样本G（x）和实际样本y。 G的目标是尽量减少这个目标对敌手D试图最大化它，即minG maxDY LGAN（G，DY，X，Y）。 我们为映射函数F：Y→X及其鉴别器DX也引入类似的对抗损失：即minF maxDX LGAN（F，DX，Y，X）。  

周期一致性损失  

敌对性训练在理论上可以学习映射G和F，分别产生同样分布的目标域Y和X（严格来说，这需要G和F是随机函数）[14]。然而，在容量足够大的情况下，网络可以将同一组输入图像映射到目标域中图像的任意随机置换，其中任何学习映射都可以导致与目标分布匹配的输出分布。因此，敌对的损失本身并不能保证所学的函数可以将单个输入xi映射到期望的输出yi。为了进一步减少可能的映射函数的空间，我们认为学习的映射函数应该是循环一致的：如图3（b）所示，对于来自域X的每个图像x，图像翻译周期应该能够使x返回到原始图像，即x→G（x）→F（G（x））≈x。我们称之为正向循环一致性。类似地，如图3（c）所示，对于来自域Y的每个图像y，G和F也应当满足反向循环一致性：y→F（y）→G（F（y））≈y。我们可以使用循环一致性损失来激励这种行为：  

公式

在初步实验中，我们也尝试用F（G（x））和x之间以及G（F（y））和y之间的对抗损失替换损失的L1范数，但没有观察到改善的性能。  

由循环一致性损失引起的行为可以在图4中观察到：重建图像F（G（x））最终与输入图像x紧密匹配。  

完整的目标  

我们的完整目标是：  

公式  

注意，我们的模型可以看作是训练两个“自动编码器”[19]：我们学习一个自动编码器F◦G：X→X与另一个G◦F：Y→Y联合。 然而，这些自动编码器每个都有特殊的内部结构：它们通过中间表示将图像映射到自身，这是将图像翻译成另一个域的过程。 这样的设置也可以被看作是“敌对自动编码器”的一个特例，它使用对抗性损失来训练自动编码器的瓶颈层以匹配任意目标分布。 在我们的例子中，X→X自动编码器的目标分配是域Y的目标分配。  

在第5.1.4节中，我们将我们的方法与消除完整目标的方法进行了比较，包括单独的对抗性损失LGAN和单独的周期一致性损失Lcyc，并且凭经验显示这两个目标在达到高质量结果中起关键作用。 我们也评估我们的方法只有一个方向的循环损失，并表明一个单一的周期是不足以规范这个under-constrained问题的训练。  

## Implementation 

网络体系结构我们改变了Johnson等人的生成网络的体系结构。 [22]谁已经显示神经风格转移和超分辨率令人印象深刻的结果。 这个网络包含两个步骤2的卷积，几个残差块[17]，以及两个分步交错的卷积，步幅为1 2。 我们对128×128图像使用6个块，对于256×256和更高分辨率训练图像使用9个块。 类似Johnson等人。 [22]，我们使用实例标准化[52]。 对于鉴别器网络，我们使用70×70PatchGANs [21,28,27]，其目的是分类70×70重叠的图像块是真实的还是假的。 这种补丁级别鉴别器体系结构比全图像鉴别器具有更少的参数，并且可以以完全卷积的方式应用于任意大小的图像[21]  

培训细节我们应用最近的两项技术来稳定我们的模型培训程序。 首先，对于LGAN（方程1），我们用最小二乘损失代替负对数似然目标[33]。 这种损失在训练期间更加稳定，并产生更高质量的结果。 特别地，对于GAN损失LGAN（G，D，X，Y），我们训练G使Ex〜pdata（x）最小化[（D（G（x）） - 1）2] Ey〜pdata（y）[（D（y）-1）2] + Ex〜pdata（x）[D（G（x））2]。  

其次，为了减少模型振荡[14]，我们遵循Shrivastava等人的策略[45]，使用生成图像的历史来更新鉴别器，而不是由最新的生成网络生成的鉴别器。 我们保留一个图像缓冲区，存储50个以前生成的图像。  

对于所有的实验，我们在等式3中设定λ= 10。我们使用批处理大小为1的Adam求解器[24]。所有网络都是从零开始训练的，学习率为0.0002。 在前100个时期，我们保持相同的学习速度，并在接下来的100个时期内将速率线性衰减到零。 有关数据集，体系结构和培训过程的更多详细信息，请参阅附录（第7章）。